# Análise das Stories

Notebook para o auxílio da análise das stories da Taís.


### Configurando jupyter e instalando as dependências do bot

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

In [2]:
# Checking versions
import rasa_nlu
import rasa_core

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


rasa_nlu: 0.14.6 rasa_core: 0.13.1


# Análise e Avaliação das Stories

### Gerando imagens de fluxo

Gerando os arquivos de imagem para a visualização e avaliação do fluxo das stories. A célula abaixo gera uma imagem para cada arquivo de stories. 
Para ver as imagens geradas acessa a pasta `img` deste notebook. As imagens são geradas em html.

In [4]:
from IPython.display import Image
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

from os import listdir
from os.path import isfile, join

agent = Agent("../../bot/domain.yml", policies=[MemoizationPolicy(), KerasPolicy()])

# Adds all stories files in a list
stories_files = [f for f in listdir("../../bot/data/stories") if isfile(join("../../bot/data/stories", f))]

# Generate the image for each file
for file in stories_files:
    
    new_img_file = './img/story_graph_' + file[:-3] + '.html'
    
    agent.visualize('../../bot/data/stories/' + file,
                    output_file = new_img_file,
                    max_history = 2)

print("Imagens salvas")

Processed Story Blocks: 100%|██████████| 53/53 [00:00<00:00, 328.01it/s, # trackers=1]


Imagens salvas


### Abrir as imagens

Para abrir todas as imagens de fluxo geradas, rode a célula abaixo, elas serão abertas em uma nova guia

In [17]:
from IPython.display import IFrame

# IFrame(src='./results/story_confmat.pdf', width=900, height=700)

for file in stories_files:
    print(file[:-3])
    display(IFrame(src='./img/story_graph_'+file[:-3]+'.html', width=900, height=700))

leirouanet


processo


main


salic


dinheiro


siglas


### Como analisar as stories

Para analisar as historias, veja como está funcionando o fluxo pelas imagens geradas, veja se está como o esperado, caso contrário deve ser feitas auterações nas stories para ajusta-lo.

O ajuste de uma história consiste na chamada das utters e intents.

### Treinando as stories e gerando o gráfico

Na celula abaixo é feito o treinamento das stories e é gerado o gráfico da matriz de confusão para a avaliação das stories. Para melhor visualização, após rodar o código, abra o arquivo `story_eval.pdf` que será gerado e salvado na pasta `img` deste notebook.

In [18]:
from rasa_core.evaluate import run_story_evaluation
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent


## Treinando modelo de diálogo
agent = Agent('../../bot/domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our neatly defined training dialogues
training_data = agent.load_data('../../bot/data/stories')

agent.train(training_data)

## salvando em models/dialogue
agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 194/194 [00:02<00:00, 67.08it/s, # trackers=18]
Processed actions: 12922it [00:41, 315.16it/s, # examples=12352]
INFO:rasa_core.policies.keras_policy:Fitting model with 12922 total samples and a validation split of 0.1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 178)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                27008     
_________________________________________________________________
dense (Dense)                (None, 93)                3069      
_________________________________________________________________
activation (Activation)      (None, 93)                0         
Total params: 30,077
Trainable params: 30,077
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12922/12922 [==============================] - 9s 676us/step - loss: 3.2905 - acc: 0.2940
Epoch 2/100
12922/12922 [==============================] - 4s 339us/step - loss: 2.6795 - acc: 0.3722
Epoch 3/100
12922/12922 [==============================] - 5s 397u

12922/12922 [==============================] - 4s 297us/step - loss: 0.0957 - acc: 0.9622
Epoch 73/100
12922/12922 [==============================] - 4s 293us/step - loss: 0.0955 - acc: 0.9608
Epoch 74/100
12922/12922 [==============================] - 4s 330us/step - loss: 0.0925 - acc: 0.9625
Epoch 75/100
12922/12922 [==============================] - 4s 317us/step - loss: 0.0955 - acc: 0.9610
Epoch 76/100
12922/12922 [==============================] - 5s 348us/step - loss: 0.0970 - acc: 0.9613
Epoch 77/100
12922/12922 [==============================] - 4s 298us/step - loss: 0.0905 - acc: 0.9619
Epoch 78/100
12922/12922 [==============================] - 4s 309us/step - loss: 0.0903 - acc: 0.9635
Epoch 79/100
12922/12922 [==============================] - 4s 328us/step - loss: 0.0901 - acc: 0.9616
Epoch 80/100
12922/12922 [==============================] - 5s 360us/step - loss: 0.0915 - acc: 0.9628
Epoch 81/100
12922/12922 [==============================] - 5s 379us/step - loss: 0.09

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to '/work/notebooks/stories/models/dialogue'


In [19]:
import sys
python = sys.executable
!{python} -m rasa_core.evaluate --core models/dialogue -s ../../bot/data/stories -o results

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-06 19:20:15.493450: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Processed Story Blocks: 100%|█| 194/194 [00:00<00:00, 1777.92it/s, # trackers=1]
INFO:__main__:Evaluating 187 stories
Progress:
100%|█████████████████████████████████████████| 187/187 [00:03<00:00, 52.38it/s]
INFO:__main__:Finished collecting predictions.
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in

INFO:rasa_nlu.evaluate:Confusion matrix, without normalization: 
[[343   0   0 ...   0   0   0]
 [  0  17   0 ...   0   0   0]
 [  0   0   2 ...   0   0   0]
 ...
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   2   0]
 [  0   0   0 ...   0   0   2]]
INFO:__main__:Finished evaluation


### Visualizar matriz de confusão
A matriz foi gerada em um pdf na pasta 'results/' para visualiza-la rode o comando abaixo

In [25]:
!ls results

failed_stories.md  story_confmat.pdf


In [27]:

display(IFrame("results/story_confmat.pdf", width=900, height=900))

## Análise da matriz de confusão

A matriz de confusão mostra as utters previstas e quais estão sendo acertadas e quais estão errando.

Na lateral da matriz está a utter a ser testada, na parte de baixo estão a utters previstas, os pontos azuis indicam qual utter está sendo prevista para cada utter analisada. A situação ideal para a matriz é uma diagonal, iniciando do canto superior esquerdo até o canto inferior direito. Pontos azuis fora dessa diagonal indicam os erros que estão ocorrendo e que devem ser tratados.

A análise também gera um arquivo `failed_stories.md` que indicam o momento em que cada story está tendo erros de previsão. Veja o arquivo na célula abaixo.

In [30]:
%cat results/failed_stories.md

## me ajuda
* cumprimentar
    - utter_cumprimentar
    - action_listen   <!-- predicted: utter_menu -->
* o_que_sei_falar
    - utter_o_que_sei_falar


## tem wpp 2
* cumprimentar
    - utter_cumprimentar
    - action_listen   <!-- predicted: utter_menu -->
* tem_wpp
    - utter_tem_wpp
    - utter_continuar_conversa


## salic 1
* cumprimentar
    - utter_cumprimentar
    - action_listen   <!-- predicted: utter_menu -->
* salic_cadastro_usuario
    - utter_salic_cadastro_usuario
    - utter_continuar_conversa


## salic 3
* cumprimentar
    - utter_cumprimentar
    - action_listen   <!-- predicted: utter_menu -->
* salic_erros
    - utter_salic_erros
    - utter_continuar_conversa


## salic 4
* cumprimentar
    - utter_cumprimentar
    - action_listen   <!-- predicted: utter_menu -->
* salic_erros_salvamento_de_proposta
    - utter_salic_erros_salvamento_de_proposta
    - utter_continuar_conversa


## salic 5
* cumprimentar
    - utter_cumprimentar
    - action_listen   <!-- predict

Na célula abaixo foi gerado um relatório a partir do arquivo `failed_stories.md`, especificando os erros que estão ocorrendo. A melhor forma de corrigi-los é tirando as inconsistencias das stories, padronizando comportamentos básicos e difenciando melhor as stories mais específicas.

In [31]:
errors = []
try:
    f = open('./results/failed_stories.md', 'r')
    lines = f.readlines()
    for i in range(len(lines)):
        lines[i] = lines[i].strip()

    for line in lines:
        if line.startswith('-'):
            l = line.split()
            utter = l[1]
            if len(l) > 2 and l[2].startswith('<!--'):
                prediction = l[4]
                if (utter,prediction) not in errors:
                    errors.append((utter, prediction))

    for e in errors:
        print('Utter com erro de previsão: {}'.format(e[0]))
        print('Utter com qual está sendo confundida: {}'.format(e[1]))
        print()

except(FileNotFoundError):
    print('Não foi encontrado nenhum erro de confusão entre as stories')

Utter com erro de previsão: action_listen
Utter com qual está sendo confundida: utter_menu

Utter com erro de previsão: utter_processo_como_funciona
Utter com qual está sendo confundida: utter_continuar_conversa

Utter com erro de previsão: utter_processo_como_funciona
Utter com qual está sendo confundida: action_listen

Utter com erro de previsão: utter_continuar_conversa
Utter com qual está sendo confundida: utter_objetivo

Utter com erro de previsão: utter_objetivo
Utter com qual está sendo confundida: utter_continuar_conversa

Utter com erro de previsão: utter_continuar_conversa
Utter com qual está sendo confundida: utter_lei_rouanet_denuncia

